In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import friedmanchisquare
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

class DieboldMarianoTest:
    """
    Implementación del test de Diebold-Mariano para comparar pronósticos
    """
    @staticmethod
    def dm_test(errors1, errors2, h=1, crit="MSE", power=2):
        """
        Realiza el test de Diebold-Mariano
        
        Parameters:
        -----------
        errors1 : array-like
            Errores del primer modelo
        errors2 : array-like
            Errores del segundo modelo
        h : int
            Horizonte de predicción (para ajustar autocorrelación)
        crit : str
            Criterio de pérdida: "MSE", "MAE", "MAPE"
        power : int
            Potencia para la función de pérdida
            
        Returns:
        --------
        dm_stat : float
            Estadístico DM
        p_value : float
            P-valor (two-tailed)
        """
        errors1 = np.array(errors1)
        errors2 = np.array(errors2)
        
        # Calcular diferencias de pérdida
        if crit == "MSE":
            loss_diff = errors1**2 - errors2**2
        elif crit == "MAE":
            loss_diff = np.abs(errors1) - np.abs(errors2)
        elif crit == "MAPE":
            loss_diff = np.abs(errors1) - np.abs(errors2)
        else:
            loss_diff = errors1**power - errors2**power
        
        # Media de las diferencias
        mean_diff = np.mean(loss_diff)
        
        # Varianza de las diferencias (ajustada por autocorrelación)
        n = len(loss_diff)
        
        # Calcular varianza con corrección de Newey-West
        gamma0 = np.var(loss_diff, ddof=1)
        
        if h > 1:
            gamma_sum = 0
            for k in range(1, h):
                gamma_k = np.cov(loss_diff[:-k], loss_diff[k:])[0, 1]
                gamma_sum += (1 - k/h) * gamma_k
            variance = (gamma0 + 2 * gamma_sum) / n
        else:
            variance = gamma0 / n
        
        # Estadístico DM
        dm_stat = mean_diff / np.sqrt(variance) if variance > 0 else 0
        
        # P-valor (two-tailed)
        p_value = 2 * (1 - stats.norm.cdf(np.abs(dm_stat)))
        
        return dm_stat, p_value


class ModelPerformanceAnalyzer:
    """
    Clase para análisis exhaustivo de rendimiento de modelos de predicción
    en diferentes escenarios de simulación.
    """
    
    def __init__(self):
        """
        Inicializa el analizador cargando los datos de los tres escenarios.
        """
        self.models = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 
                      'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']
        
        # Cargar datos con las rutas especificadas
        print("Cargando datos...")
        
        try:
            self.df_estacionario = pd.read_excel("./Datos/estacionario.xlsx")
            self.df_estacionario['Escenario'] = 'Estacionario_Lineal'
            print(f"✓ Estacionario: {len(self.df_estacionario)} filas")
            print(f"  Columnas: {self.df_estacionario.columns.tolist()}")
            
            self.df_no_estacionario = pd.read_excel("./Datos/no_estacionario.xlsx")
            self.df_no_estacionario['Escenario'] = 'No_Estacionario_Lineal'
            print(f"✓ No Estacionario: {len(self.df_no_estacionario)} filas")
            print(f"  Columnas: {self.df_no_estacionario.columns.tolist()}")
            
            self.df_no_lineal = pd.read_excel("./Datos/no_lineal.xlsx")
            self.df_no_lineal['Escenario'] = 'No_Lineal'
            print(f"✓ No Lineal: {len(self.df_no_lineal)} filas")
            print(f"  Columnas: {self.df_no_lineal.columns.tolist()}")
            
        except FileNotFoundError as e:
            print(f"ERROR: No se encontró el archivo - {e}")
            print("Verifica que los archivos estén en la carpeta './Datos/'")
            raise
        
        # Estandarizar nombres de columnas
        self._standardize_columns()
        
        # Combinar todos los datos
        self.df_all = pd.concat([self.df_estacionario, self.df_no_estacionario, 
                                 self.df_no_lineal], ignore_index=True)
        
        # Convertir tipos de datos críticos
        self._convert_data_types()
        
        print(f"\n✓ Datos combinados: {len(self.df_all)} observaciones totales")
        print(f"✓ Columnas finales: {self.df_all.columns.tolist()}")
        
    def _standardize_columns(self):
        """Estandariza nombres de columnas entre datasets"""
        # Para estacionario
        if 'Varianza error' in self.df_estacionario.columns:
            self.df_estacionario.rename(columns={'Varianza error': 'Varianza'}, inplace=True)
        
        # Agregar columna 'Tipo de Modelo' si no existe en estacionario
        if 'Tipo de Modelo' not in self.df_estacionario.columns:
            # Crear tipo de modelo basado en valores AR y MA
            def create_model_type(row):
                ar_vals = row.get('Valores de AR', '')
                ma_vals = row.get('Valores MA', '')
                
                ar_str = str(ar_vals) if pd.notna(ar_vals) else ''
                ma_str = str(ma_vals) if pd.notna(ma_vals) else ''
                
                # Contar órdenes
                ar_order = len([x for x in ar_str.split(',') if x.strip() and x.strip() != '[]']) if ar_str else 0
                ma_order = len([x for x in ma_str.split(',') if x.strip() and x.strip() != '[]']) if ma_str else 0
                
                if ar_order > 0 and ma_order > 0:
                    return f'ARMA({ar_order},{ma_order})'
                elif ar_order > 0:
                    return f'AR({ar_order})'
                elif ma_order > 0:
                    return f'MA({ma_order})'
                else:
                    return 'Unknown'
            
            self.df_estacionario['Tipo de Modelo'] = self.df_estacionario.apply(create_model_type, axis=1)
        
        # Para no estacionario
        if 'Varianza error' in self.df_no_estacionario.columns:
            self.df_no_estacionario.rename(columns={'Varianza error': 'Varianza'}, inplace=True)
        
        # Para no lineal
        if 'Varianza error' in self.df_no_lineal.columns:
            self.df_no_lineal.rename(columns={'Varianza error': 'Varianza'}, inplace=True)
    
    def _convert_data_types(self):
        """Convierte tipos de datos para evitar errores de comparación"""
        # Convertir 'Paso' a numérico
        self.df_all['Paso'] = pd.to_numeric(self.df_all['Paso'], errors='coerce')
        
        # Convertir 'Varianza' a numérico
        self.df_all['Varianza'] = pd.to_numeric(self.df_all['Varianza'], errors='coerce')
        
        # Convertir columnas de modelos a numérico
        for model in self.models:
            self.df_all[model] = pd.to_numeric(self.df_all[model], errors='coerce')
        
        # Eliminar filas con valores NaN críticos
        critical_cols = ['Paso', 'Varianza'] + self.models
        self.df_all.dropna(subset=critical_cols, inplace=True)
        
        print(f"✓ Tipos de datos convertidos")
        print(f"✓ Filas después de limpieza: {len(self.df_all)}")
        
    def generate_full_report(self, output_dir='resultados_analisis'):
        """
        Genera reporte completo respondiendo a todas las preguntas clave.
        """
        import os
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        print("\n" + "="*80)
        print("INICIANDO ANÁLISIS COMPREHENSIVO DE MODELOS")
        print("="*80)
        
        # Crear archivo de reporte
        report_file = f"{output_dir}/reporte_completo.txt"
        with open(report_file, 'w', encoding='utf-8') as f:
            f.write("REPORTE COMPLETO DE ANÁLISIS DE MODELOS DE PREDICCIÓN\n")
            f.write("="*80 + "\n\n")
        
        # 1. ANÁLISIS POR CARACTERÍSTICAS DEL DGP
        print("\n1. Analizando características del proceso generador...")
        self.analyze_dgp_characteristics(output_dir)
        
        # 2. ANÁLISIS POR DISTRIBUCIÓN DE ERRORES
        print("\n2. Analizando efecto de distribuciones...")
        self.analyze_distribution_effects(output_dir)
        
        # 3. ANÁLISIS POR HORIZONTE DE PREDICCIÓN
        print("\n3. Analizando horizonte de predicción...")
        self.analyze_horizon_effects(output_dir)
        
        # 4. ANÁLISIS DE INTERACCIONES COMPLEJAS
        print("\n4. Analizando interacciones complejas...")
        self.analyze_interactions(output_dir)
        
        # 5. ANÁLISIS DE ROBUSTEZ Y ESTABILIDAD
        print("\n5. Analizando robustez y estabilidad...")
        self.analyze_robustness(output_dir)
        
        # 6. ANÁLISIS DE SIGNIFICANCIA ESTADÍSTICA (DIEBOLD-MARIANO)
        print("\n6. Realizando tests de Diebold-Mariano...")
        self.analyze_statistical_significance_dm(output_dir)
        
        # 7. ANÁLISIS POR MODELO INDIVIDUAL
        print("\n7. Generando perfiles por modelo...")
        self.analyze_individual_models(output_dir)
        
        # 8. RECOMENDACIONES Y CONCLUSIONES
        print("\n8. Generando recomendaciones...")
        self.generate_recommendations(output_dir)
        
        print(f"\n{'='*80}")
        print(f"ANÁLISIS COMPLETO. Resultados guardados en: {output_dir}/")
        print(f"{'='*80}")
        
    def analyze_dgp_characteristics(self, output_dir):
        """
        1. ANÁLISIS DE CARACTERÍSTICAS DEL PROCESO GENERADOR
        """
        results = []
        
        # 1.1 Efecto de estacionaridad
        print("  - Analizando efecto de estacionaridad...")
        for model in self.models:
            est_mean = self.df_estacionario[model].mean()
            no_est_mean = self.df_no_estacionario[model].mean()
            diff = no_est_mean - est_mean
            pct_change = (diff / est_mean) * 100 if est_mean != 0 else 0
            
            results.append({
                'Modelo': model,
                'ECRPS_Estacionario': est_mean,
                'ECRPS_No_Estacionario': no_est_mean,
                'Diferencia': diff,
                'Cambio_%': pct_change
            })
        
        df_estacionaridad = pd.DataFrame(results)
        df_estacionaridad = df_estacionaridad.sort_values('Cambio_%')
        df_estacionaridad.to_csv(f'{output_dir}/1_efecto_estacionaridad.csv', index=False)
        
        # Visualización
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # Gráfico de barras comparativas
        x = np.arange(len(self.models))
        width = 0.35
        axes[0].bar(x - width/2, df_estacionaridad['ECRPS_Estacionario'], 
                   width, label='Estacionario', alpha=0.8)
        axes[0].bar(x + width/2, df_estacionaridad['ECRPS_No_Estacionario'], 
                   width, label='No Estacionario', alpha=0.8)
        axes[0].set_xlabel('Modelo')
        axes[0].set_ylabel('ECRPS Promedio')
        axes[0].set_title('Rendimiento: Estacionario vs No Estacionario')
        axes[0].set_xticks(x)
        axes[0].set_xticklabels(df_estacionaridad['Modelo'], rotation=45, ha='right')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Gráfico de cambio porcentual
        colors = ['green' if x < 0 else 'red' for x in df_estacionaridad['Cambio_%']]
        axes[1].barh(df_estacionaridad['Modelo'], df_estacionaridad['Cambio_%'], color=colors, alpha=0.7)
        axes[1].set_xlabel('Cambio Porcentual (%)')
        axes[1].set_title('Impacto de No Estacionaridad\n(Negativo = Mejor en No Estacionario)')
        axes[1].axvline(x=0, color='black', linestyle='--', linewidth=0.8)
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/1_estacionaridad.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # 1.2 Efecto de no linealidad
        print("  - Analizando efecto de no linealidad...")
        results_nl = []
        for model in self.models:
            lin_mean = self.df_estacionario[model].mean()
            nl_mean = self.df_no_lineal[model].mean()
            diff = nl_mean - lin_mean
            pct_change = (diff / lin_mean) * 100 if lin_mean != 0 else 0
            
            results_nl.append({
                'Modelo': model,
                'ECRPS_Lineal': lin_mean,
                'ECRPS_No_Lineal': nl_mean,
                'Diferencia': diff,
                'Cambio_%': pct_change
            })
        
        df_linealidad = pd.DataFrame(results_nl)
        df_linealidad = df_linealidad.sort_values('Cambio_%')
        df_linealidad.to_csv(f'{output_dir}/1_efecto_no_linealidad.csv', index=False)
        
        # Visualización no linealidad
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        x = np.arange(len(self.models))
        axes[0].bar(x - width/2, df_linealidad['ECRPS_Lineal'], 
                   width, label='Lineal', alpha=0.8)
        axes[0].bar(x + width/2, df_linealidad['ECRPS_No_Lineal'], 
                   width, label='No Lineal', alpha=0.8)
        axes[0].set_xlabel('Modelo')
        axes[0].set_ylabel('ECRPS Promedio')
        axes[0].set_title('Rendimiento: Lineal vs No Lineal')
        axes[0].set_xticks(x)
        axes[0].set_xticklabels(df_linealidad['Modelo'], rotation=45, ha='right')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        colors = ['green' if x < 0 else 'red' for x in df_linealidad['Cambio_%']]
        axes[1].barh(df_linealidad['Modelo'], df_linealidad['Cambio_%'], color=colors, alpha=0.7)
        axes[1].set_xlabel('Cambio Porcentual (%)')
        axes[1].set_title('Impacto de No Linealidad\n(Negativo = Mejor en No Lineal)')
        axes[1].axvline(x=0, color='black', linestyle='--', linewidth=0.8)
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/1_no_linealidad.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # 1.3 Análisis por tipo de modelo
        print("  - Analizando efecto del tipo de modelo...")
        self.analyze_model_type_effect(output_dir)
        
    def analyze_model_type_effect(self, output_dir):
        """Analiza el efecto del tipo de modelo en el rendimiento"""
        
        # Análisis para datos estacionarios
        if 'Tipo de Modelo' in self.df_estacionario.columns:
            results_type = []
            for model in self.models:
                for model_type in self.df_estacionario['Tipo de Modelo'].unique():
                    subset = self.df_estacionario[self.df_estacionario['Tipo de Modelo'] == model_type]
                    if len(subset) > 0:
                        results_type.append({
                            'Modelo_Predictor': model,
                            'Tipo_Proceso': model_type,
                            'ECRPS_Mean': subset[model].mean(),
                            'ECRPS_Std': subset[model].std(),
                            'N_Obs': len(subset)
                        })
            
            df_type = pd.DataFrame(results_type)
            df_type.to_csv(f'{output_dir}/1_efecto_tipo_modelo.csv', index=False)
            
            # Crear heatmap para tipos más comunes
            common_types = df_type['Tipo_Proceso'].value_counts().head(10).index
            df_type_filtered = df_type[df_type['Tipo_Proceso'].isin(common_types)]
            
            if len(df_type_filtered) > 0:
                pivot = df_type_filtered.pivot_table(
                    index='Modelo_Predictor', 
                    columns='Tipo_Proceso', 
                    values='ECRPS_Mean'
                )
                
                fig, ax = plt.subplots(figsize=(14, 8))
                sns.heatmap(pivot, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=ax, 
                           cbar_kws={'label': 'ECRPS'})
                ax.set_title('Rendimiento por Modelo Predictor y Tipo de Proceso', fontsize=14)
                ax.set_xlabel('Tipo de Proceso')
                ax.set_ylabel('Modelo Predictor')
                plt.tight_layout()
                plt.savefig(f'{output_dir}/1_heatmap_tipo_modelo.png', dpi=300, bbox_inches='tight')
                plt.close()
        
    def analyze_distribution_effects(self, output_dir):
        """
        2. ANÁLISIS DE EFECTOS DE DISTRIBUCIÓN
        """
        print("  - Analizando efectos de distribuciones...")
        
        results_dist = []
        for model in self.models:
            for dist in self.df_all['Distribución'].unique():
                if pd.notna(dist):
                    subset = self.df_all[self.df_all['Distribución'] == dist]
                    if len(subset) > 0:
                        results_dist.append({
                            'Modelo': model,
                            'Distribución': dist,
                            'ECRPS_Mean': subset[model].mean(),
                            'ECRPS_Std': subset[model].std(),
                            'ECRPS_Min': subset[model].min(),
                            'ECRPS_Max': subset[model].max()
                        })
        
        df_dist = pd.DataFrame(results_dist)
        df_dist.to_csv(f'{output_dir}/2_efecto_distribucion.csv', index=False)
        
        # Heatmap
        if len(df_dist) > 0:
            pivot = df_dist.pivot(index='Modelo', columns='Distribución', values='ECRPS_Mean')
            
            fig, ax = plt.subplots(figsize=(10, 8))
            sns.heatmap(pivot, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=ax, cbar_kws={'label': 'ECRPS'})
            ax.set_title('Rendimiento por Modelo y Distribución', fontsize=14)
            plt.tight_layout()
            plt.savefig(f'{output_dir}/2_heatmap_distribucion.png', dpi=300, bbox_inches='tight')
            plt.close()
        
        # Análisis por varianza
        print("  - Analizando efectos de varianza...")
        results_var = []
        varianzas_unicas = sorted([v for v in self.df_all['Varianza'].unique() if pd.notna(v)])
        
        for model in self.models:
            for var in varianzas_unicas:
                subset = self.df_all[self.df_all['Varianza'] == var]
                if len(subset) > 0:
                    results_var.append({
                        'Modelo': model,
                        'Varianza': var,
                        'ECRPS_Mean': subset[model].mean(),
                        'ECRPS_Std': subset[model].std()
                    })
        
        df_var = pd.DataFrame(results_var)
        df_var.to_csv(f'{output_dir}/2_efecto_varianza.csv', index=False)
        
        # Gráfico de líneas por varianza
        if len(df_var) > 0:
            fig, ax = plt.subplots(figsize=(12, 8))
            for model in self.models:
                data = df_var[df_var['Modelo'] == model].sort_values('Varianza')
                if len(data) > 0:
                    ax.plot(data['Varianza'], data['ECRPS_Mean'], marker='o', label=model, linewidth=2)
            
            ax.set_xlabel('Varianza', fontsize=12)
            ax.set_ylabel('ECRPS Promedio', fontsize=12)
            ax.set_title('Rendimiento según Nivel de Varianza', fontsize=14)
            ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            ax.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.savefig(f'{output_dir}/2_efecto_varianza.png', dpi=300, bbox_inches='tight')
            plt.close()
        
    def analyze_horizon_effects(self, output_dir):
        """
        3. ANÁLISIS DE HORIZONTE DE PREDICCIÓN
        """
        print("  - Analizando deterioro por horizonte...")
        
        results_horizon = []
        pasos_unicos = sorted([p for p in self.df_all['Paso'].unique() if pd.notna(p)])
        
        for model in self.models:
            for paso in pasos_unicos:
                subset = self.df_all[self.df_all['Paso'] == paso]
                if len(subset) > 0:
                    mean_val = subset[model].mean()
                    std_val = subset[model].std()
                    cv_val = std_val / mean_val if mean_val != 0 and pd.notna(mean_val) else 0
                    
                    results_horizon.append({
                        'Modelo': model,
                        'Paso': int(paso),
                        'ECRPS_Mean': mean_val,
                        'ECRPS_Std': std_val,
                        'ECRPS_CV': cv_val
                    })
        
        df_horizon = pd.DataFrame(results_horizon)
        df_horizon.to_csv(f'{output_dir}/3_efecto_horizonte.csv', index=False)
        
        # Gráfico de deterioro
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        
        # ECRPS promedio por paso
        for model in self.models:
            data = df_horizon[df_horizon['Modelo'] == model].sort_values('Paso')
            if len(data) > 0:
                axes[0].plot(data['Paso'], data['ECRPS_Mean'], marker='o', label=model, linewidth=2)
        
        axes[0].set_xlabel('Paso de Predicción', fontsize=12)
        axes[0].set_ylabel('ECRPS Promedio', fontsize=12)
        axes[0].set_title('Deterioro del Rendimiento por Horizonte', fontsize=14)
        axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        axes[0].grid(True, alpha=0.3)
        
        # Tasa de deterioro
        deterioro = []
        for model in self.models:
            data = df_horizon[df_horizon['Modelo'] == model].sort_values('Paso')
            if len(data) >= 2:
                paso_values = data['Paso'].tolist()
                ecrps_paso1 = data.iloc[0]['ECRPS_Mean']
                ecrps_paso_final = data.iloc[-1]['ECRPS_Mean']
                
                if pd.notna(ecrps_paso1) and pd.notna(ecrps_paso_final) and ecrps_paso1 != 0:
                    tasa = ((ecrps_paso_final - ecrps_paso1) / ecrps_paso1) * 100
                    deterioro.append({'Modelo': model, 'Deterioro_%': tasa})
        
        if deterioro:
            df_deterioro = pd.DataFrame(deterioro).sort_values('Deterioro_%')
            colors = ['green' if x < df_deterioro['Deterioro_%'].median() else 'red' 
                     for x in df_deterioro['Deterioro_%']]
            axes[1].barh(df_deterioro['Modelo'], df_deterioro['Deterioro_%'], color=colors, alpha=0.7)
            axes[1].set_xlabel(f'Deterioro Paso {pasos_unicos[0]}→{pasos_unicos[-1]} (%)', fontsize=12)
            axes[1].set_title('Tasa de Deterioro por Modelo', fontsize=14)
            axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/3_horizonte_prediccion.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Análisis de consistencia de ranking
        print("  - Analizando consistencia de ranking...")
        ranking_consistency = []
        for paso in pasos_unicos:
            subset = self.df_all[self.df_all['Paso'] == paso]
            if len(subset) > 0:
                ranks = subset[self.models].mean().rank()
                rank_dict = ranks.to_dict()
                rank_dict['Paso'] = int(paso)
                ranking_consistency.append(rank_dict)
        
        df_ranks = pd.DataFrame(ranking_consistency)
        df_ranks.to_csv(f'{output_dir}/3_ranking_por_paso.csv', index=False)
        
    def analyze_interactions(self, output_dir):
        """
        4. ANÁLISIS DE INTERACCIONES COMPLEJAS
        """
        print("  - Analizando interacciones Escenario × Distribución...")
        
        results_int = []
        for model in self.models:
            for escenario in self.df_all['Escenario'].unique():
                for dist in self.df_all['Distribución'].unique():
                    subset = self.df_all[(self.df_all['Escenario'] == escenario) & 
                                        (self.df_all['Distribución'] == dist)]
                    if len(subset) > 0:
                        results_int.append({
                            'Modelo': model,
                            'Escenario': escenario,
                            'Distribución': dist,
                            'ECRPS_Mean': subset[model].mean()
                        })
        
        df_int = pd.DataFrame(results_int)
        df_int.to_csv(f'{output_dir}/4_interacciones.csv', index=False)
        
        # Heatmap de interacciones para cada modelo
        for model in self.models[:3]:  # Solo primeros 3 por espacio
            model_data = df_int[df_int['Modelo'] == model]
            if len(model_data) > 0:
                pivot = model_data.pivot(
                    index='Escenario', columns='Distribución', values='ECRPS_Mean')
                
                fig, ax = plt.subplots(figsize=(10, 6))
                sns.heatmap(pivot, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=ax)
                ax.set_title(f'Interacción Escenario × Distribución: {model}', fontsize=12)
                plt.tight_layout()
                plt.savefig(f'{output_dir}/4_interaccion_{model.replace(" ", "_")}.png', 
                           dpi=300, bbox_inches='tight')
                plt.close()
        
        # Interacción triple: Escenario × Varianza × Paso
        print("  - Analizando interacción triple...")
        results_triple = []
        
        varianzas_unicas = sorted([v for v in self.df_all['Varianza'].unique() if pd.notna(v)])
        pasos_unicos = sorted([p for p in self.df_all['Paso'].unique() if pd.notna(p)])
        
        for model in self.models:
            for escenario in self.df_all['Escenario'].unique():
                for var in varianzas_unicas:
                    for paso in pasos_unicos:
                        subset = self.df_all[
                            (self.df_all['Escenario'] == escenario) & 
                            (self.df_all['Varianza'] == var) &
                            (self.df_all['Paso'] == paso)
                        ]
                        if len(subset) > 0:
                            results_triple.append({
                                'Modelo': model,
                                'Escenario': escenario,
                                'Varianza': var,
                                'Paso': int(paso),
                                'ECRPS_Mean': subset[model].mean()
                            })
        
        df_triple = pd.DataFrame(results_triple)
        df_triple.to_csv(f'{output_dir}/4_interaccion_triple.csv', index=False)
        
    def analyze_robustness(self, output_dir):
        """
        5. ANÁLISIS DE ROBUSTEZ Y ESTABILIDAD
        """
        print("  - Calculando métricas de robustez...")
        
        results_robust = []
        for model in self.models:
            ecrps_values = self.df_all[model]
            
            results_robust.append({
                'Modelo': model,
                'ECRPS_Mean': ecrps_values.mean(),
                'ECRPS_Std': ecrps_values.std(),
                'ECRPS_CV': ecrps_values.std() / ecrps_values.mean() if ecrps_values.mean() != 0 else 0,
                'ECRPS_Min': ecrps_values.min(),
                'ECRPS_Q25': ecrps_values.quantile(0.25),
                'ECRPS_Median': ecrps_values.median(),
                'ECRPS_Q75': ecrps_values.quantile(0.75),
                'ECRPS_Max': ecrps_values.max(),
                'ECRPS_IQR': ecrps_values.quantile(0.75) - ecrps_values.quantile(0.25)
            })
        
        df_robust = pd.DataFrame(results_robust)
        df_robust = df_robust.sort_values('ECRPS_CV')
        df_robust.to_csv(f'{output_dir}/5_robustez.csv', index=False)
        
        # Gráfico de robustez
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Coeficiente de variación
        axes[0, 0].barh(df_robust['Modelo'], df_robust['ECRPS_CV'], alpha=0.7)
        axes[0, 0].set_xlabel('Coeficiente de Variación')
        axes[0, 0].set_title('Estabilidad (Menor CV = Más Estable)')
        axes[0, 0].grid(True, alpha=0.3)
        
        # Rango intercuartílico
        axes[0, 1].barh(df_robust['Modelo'], df_robust['ECRPS_IQR'], alpha=0.7, color='coral')
        axes[0, 1].set_xlabel('Rango Intercuartílico')
        axes[0, 1].set_title('Variabilidad (Menor IQR = Más Consistente)')
        axes[0, 1].grid(True, alpha=0.3)
        
        # Boxplot comparativo
        data_box = [self.df_all[model] for model in self.models]
        bp = axes[1, 0].boxplot(data_box, labels=self.models, patch_artist=True)
        for patch in bp['boxes']:
            patch.set_facecolor('lightblue')
        axes[1, 0].set_ylabel('ECRPS')
        axes[1, 0].set_title('Distribución de ECRPS por Modelo')
        axes[1, 0].tick_params(axis='x', rotation=45)
        axes[1, 0].grid(True, alpha=0.3)
        
        # Scatter: Media vs Variabilidad
        axes[1, 1].scatter(df_robust['ECRPS_Mean'], df_robust['ECRPS_Std'], 
                          s=100, alpha=0.6, c=range(len(df_robust)), cmap='viridis')
        for idx, row in df_robust.iterrows():
            axes[1, 1].annotate(row['Modelo'], 
                               (row['ECRPS_Mean'], row['ECRPS_Std']),
                               fontsize=8, alpha=0.7)
        axes[1, 1].set_xlabel('ECRPS Promedio')
        axes[1, 1].set_ylabel('Desviación Estándar')
        axes[1, 1].set_title('Trade-off Rendimiento vs Estabilidad')
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/5_robustez.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Análisis de peores casos
        print("  - Identificando peores casos...")
        worst_cases = []
        for model in self.models:
            df_temp = self.df_all.copy()
            df_temp['ECRPS'] = df_temp[model]
            worst = df_temp.nlargest(10, 'ECRPS')[
                ['Escenario', 'Tipo de Modelo', 'Distribución', 'Varianza', 'Paso', 'ECRPS']
            ]
            worst['Modelo_Predictor'] = model
            worst_cases.append(worst)
        
        df_worst = pd.concat(worst_cases, ignore_index=True)
        df_worst.to_csv(f'{output_dir}/5_peores_casos.csv', index=False)
        
    def analyze_statistical_significance_dm(self, output_dir):
        """
        6. ANÁLISIS DE SIGNIFICANCIA ESTADÍSTICA CON DIEBOLD-MARIANO
        """
        print("  - Realizando tests de Diebold-Mariano...")
        
        # Test de Friedman por escenario (para comparación general)
        results_friedman = []
        for escenario in self.df_all['Escenario'].unique():
            subset = self.df_all[self.df_all['Escenario'] == escenario]
            data_matrix = subset[self.models].values
            
            try:
                statistic, p_value = friedmanchisquare(*[data_matrix[:, i] for i in range(len(self.models))])
                
                results_friedman.append({
                    'Escenario': escenario,
                    'Friedman_Statistic': statistic,
                    'P_Value': p_value,
                    'Significativo': 'Sí' if p_value < 0.05 else 'No'
                })
            except Exception as e:
                print(f"    Advertencia: Error en test de Friedman para {escenario}: {e}")
        
        if results_friedman:
            df_friedman = pd.DataFrame(results_friedman)
            df_friedman.to_csv(f'{output_dir}/6_test_friedman.csv', index=False)
        
        # Tests de Diebold-Mariano pareados
        print("  - Realizando tests pareados de Diebold-Mariano...")
        pairs = list(combinations(self.models, 2))
        dm_results = []
        
        for model1, model2 in pairs:
            # Calcular errores (usamos ECRPS directamente como métrica de pérdida)
            errors1 = self.df_all[model1].values
            errors2 = self.df_all[model2].values
            
            # Test de Diebold-Mariano
            dm_stat, p_value = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
            
            mean_diff = self.df_all[model1].mean() - self.df_all[model2].mean()
            
            # Determinar ganador
            if p_value < 0.05:
                if mean_diff < 0:
                    ganador = model1
                else:
                    ganador = model2
            else:
                ganador = 'Empate'
            
            dm_results.append({
                'Modelo_1': model1,
                'Modelo_2': model2,
                'Diferencia_Media': mean_diff,
                'DM_Statistic': dm_stat,
                'P_Value': p_value,
                'Significativo_0.05': 'Sí' if p_value < 0.05 else 'No',
                'Significativo_0.01': 'Sí' if p_value < 0.01 else 'No',
                'Ganador': ganador
            })
        
        df_dm = pd.DataFrame(dm_results)
        df_dm = df_dm.sort_values('P_Value')
        df_dm.to_csv(f'{output_dir}/6_tests_diebold_mariano.csv', index=False)
        
        # Matriz de p-valores (Diebold-Mariano)
        print("  - Creando matriz de p-valores...")
        p_matrix = np.ones((len(self.models), len(self.models)))
        for i, model1 in enumerate(self.models):
            for j, model2 in enumerate(self.models):
                if i != j:
                    errors1 = self.df_all[model1].values
                    errors2 = self.df_all[model2].values
                    _, p_val = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
                    p_matrix[i, j] = p_val
        
        fig, ax = plt.subplots(figsize=(12, 10))
        sns.heatmap(p_matrix, annot=True, fmt='.3f', cmap='RdYlGn', 
                   xticklabels=self.models, yticklabels=self.models, 
                   ax=ax, vmin=0, vmax=0.1, cbar_kws={'label': 'P-valor'})
        ax.set_title('Matriz de P-valores (Test de Diebold-Mariano)\nVerde = Diferencia Significativa', 
                    fontsize=14)
        plt.tight_layout()
        plt.savefig(f'{output_dir}/6_matriz_pvalores_dm.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Dominancia estadística con Diebold-Mariano
        print("  - Analizando dominancia estadística...")
        dominance = []
        for model in self.models:
            wins = 0
            losses = 0
            ties = 0
            for other_model in self.models:
                if model != other_model:
                    errors1 = self.df_all[model].values
                    errors2 = self.df_all[other_model].values
                    _, p_val = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
                    mean_diff = self.df_all[model].mean() - self.df_all[other_model].mean()
                    
                    if p_val < 0.05:
                        if mean_diff < 0:  # modelo es mejor (menor ECRPS)
                            wins += 1
                        else:
                            losses += 1
                    else:
                        ties += 1
            
            dominance.append({
                'Modelo': model,
                'Victorias_Significativas': wins,
                'Derrotas_Significativas': losses,
                'Empates': ties,
                'Score_Neto': wins - losses
            })
        
        df_dominance = pd.DataFrame(dominance)
        df_dominance = df_dominance.sort_values('Score_Neto', ascending=False)
        df_dominance.to_csv(f'{output_dir}/6_dominancia_estadistica_dm.csv', index=False)
        
        # Visualización de dominancia
        fig, ax = plt.subplots(figsize=(12, 6))
        x = np.arange(len(df_dominance))
        width = 0.25
        
        ax.bar(x - width, df_dominance['Victorias_Significativas'], 
               width, label='Victorias', color='green', alpha=0.7)
        ax.bar(x, df_dominance['Empates'], 
               width, label='Empates', color='gray', alpha=0.7)
        ax.bar(x + width, df_dominance['Derrotas_Significativas'], 
               width, label='Derrotas', color='red', alpha=0.7)
        
        ax.set_xlabel('Modelo')
        ax.set_ylabel('Número de Comparaciones')
        ax.set_title('Dominancia Estadística (Test Diebold-Mariano)')
        ax.set_xticks(x)
        ax.set_xticklabels(df_dominance['Modelo'], rotation=45, ha='right')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'{output_dir}/6_dominancia_dm.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Análisis de Diebold-Mariano por escenario
        print("  - Analizando DM por escenario...")
        dm_by_scenario = []
        for escenario in self.df_all['Escenario'].unique():
            subset = self.df_all[self.df_all['Escenario'] == escenario]
            
            for model1, model2 in combinations(self.models, 2):
                errors1 = subset[model1].values
                errors2 = subset[model2].values
                
                if len(errors1) > 0 and len(errors2) > 0:
                    dm_stat, p_value = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
                    mean_diff = subset[model1].mean() - subset[model2].mean()
                    
                    dm_by_scenario.append({
                        'Escenario': escenario,
                        'Modelo_1': model1,
                        'Modelo_2': model2,
                        'DM_Statistic': dm_stat,
                        'P_Value': p_value,
                        'Diferencia_Media': mean_diff,
                        'Significativo': 'Sí' if p_value < 0.05 else 'No'
                    })
        
        df_dm_scenario = pd.DataFrame(dm_by_scenario)
        df_dm_scenario.to_csv(f'{output_dir}/6_dm_por_escenario.csv', index=False)
    
    def analyze_individual_models(self, output_dir):
        """
        7. PERFILES INDIVIDUALES POR MODELO
        """
        print("  - Generando perfiles individuales...")
        
        for model in self.models:
            print(f"    > Analizando {model}...")
            
            # Crear subdirectorio para el modelo
            model_dir = f"{output_dir}/perfiles_modelos/{model.replace(' ', '_')}"
            import os
            os.makedirs(model_dir, exist_ok=True)
            
            # Reporte del modelo
            report = []
            report.append(f"="*80)
            report.append(f"PERFIL DETALLADO: {model}")
            report.append(f"="*80)
            report.append("")
            
            # Estadísticas generales
            report.append("1. ESTADÍSTICAS GENERALES")
            report.append("-" * 40)
            report.append(f"ECRPS Promedio Global: {self.df_all[model].mean():.6f}")
            report.append(f"Desviación Estándar: {self.df_all[model].std():.6f}")
            cv = self.df_all[model].std()/self.df_all[model].mean() if self.df_all[model].mean() != 0 else 0
            report.append(f"Coeficiente de Variación: {cv:.4f}")
            report.append(f"Mínimo: {self.df_all[model].min():.6f}")
            report.append(f"Mediana: {self.df_all[model].median():.6f}")
            report.append(f"Máximo: {self.df_all[model].max():.6f}")
            report.append("")
            
            # Ranking general
            mean_scores = self.df_all[self.models].mean()
            ranking = mean_scores.rank().astype(int)
            report.append(f"Ranking General: {ranking[model]}° de {len(self.models)}")
            report.append("")
            
            # Mejor escenario
            report.append("2. MEJORES ESCENARIOS")
            report.append("-" * 40)
            best_idx = self.df_all[model].idxmin()
            best_row = self.df_all.loc[best_idx]
            report.append(f"Mejor ECRPS: {best_row[model]:.6f}")
            report.append(f"  - Escenario: {best_row['Escenario']}")
            if 'Tipo de Modelo' in best_row:
                report.append(f"  - Tipo Modelo: {best_row['Tipo de Modelo']}")
            report.append(f"  - Distribución: {best_row['Distribución']}")
            report.append(f"  - Varianza: {best_row['Varianza']}")
            report.append(f"  - Paso: {best_row['Paso']}")
            report.append("")
            
            # Peor escenario
            report.append("3. PEORES ESCENARIOS")
            report.append("-" * 40)
            worst_idx = self.df_all[model].idxmax()
            worst_row = self.df_all.loc[worst_idx]
            report.append(f"Peor ECRPS: {worst_row[model]:.6f}")
            report.append(f"  - Escenario: {worst_row['Escenario']}")
            if 'Tipo de Modelo' in worst_row:
                report.append(f"  - Tipo Modelo: {worst_row['Tipo de Modelo']}")
            report.append(f"  - Distribución: {worst_row['Distribución']}")
            report.append(f"  - Varianza: {worst_row['Varianza']}")
            report.append(f"  - Paso: {worst_row['Paso']}")
            report.append("")
            
            # Rendimiento por escenario
            report.append("4. RENDIMIENTO POR ESCENARIO")
            report.append("-" * 40)
            for escenario in ['Estacionario_Lineal', 'No_Estacionario_Lineal', 'No_Lineal']:
                subset = self.df_all[self.df_all['Escenario'] == escenario]
                if len(subset) > 0:
                    mean_val = subset[model].mean()
                    rank = subset[self.models].mean().rank()[model]
                    report.append(f"{escenario}:")
                    report.append(f"  ECRPS: {mean_val:.6f} (Ranking: {int(rank)}°)")
            report.append("")
            
            # Fortalezas y debilidades
            report.append("5. FORTALEZAS Y DEBILIDADES")
            report.append("-" * 40)
            
            # Por distribución
            report.append("Por Distribución:")
            dist_performance = []
            for dist in self.df_all['Distribución'].unique():
                subset = self.df_all[self.df_all['Distribución'] == dist]
                if len(subset) > 0:
                    mean_val = subset[model].mean()
                    rank = subset[self.models].mean().rank()[model]
                    dist_performance.append((dist, mean_val, rank))
            
            if dist_performance:
                dist_performance.sort(key=lambda x: x[2])
                report.append(f"  Mejor: {dist_performance[0][0]} (Ranking {int(dist_performance[0][2])}°)")
                report.append(f"  Peor: {dist_performance[-1][0]} (Ranking {int(dist_performance[-1][2])}°)")
            report.append("")
            
            # Por varianza
            report.append("Por Varianza:")
            var_performance = []
            for var in sorted(self.df_all['Varianza'].unique()):
                subset = self.df_all[self.df_all['Varianza'] == var]
                if len(subset) > 0:
                    mean_val = subset[model].mean()
                    rank = subset[self.models].mean().rank()[model]
                    var_performance.append((var, mean_val, rank))
            
            if var_performance:
                var_performance.sort(key=lambda x: x[2])
                report.append(f"  Mejor: Varianza {var_performance[0][0]} (Ranking {int(var_performance[0][2])}°)")
                report.append(f"  Peor: Varianza {var_performance[-1][0]} (Ranking {int(var_performance[-1][2])}°)")
            report.append("")
            
            # Comparaciones con Diebold-Mariano
            report.append("6. COMPARACIONES ESTADÍSTICAS (DIEBOLD-MARIANO)")
            report.append("-" * 40)
            
            wins = 0
            losses = 0
            for other_model in self.models:
                if model != other_model:
                    errors1 = self.df_all[model].values
                    errors2 = self.df_all[other_model].values
                    _, p_val = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
                    mean_diff = self.df_all[model].mean() - self.df_all[other_model].mean()
                    
                    if p_val < 0.05:
                        if mean_diff < 0:
                            wins += 1
                        else:
                            losses += 1
            
            report.append(f"Victorias significativas: {wins}")
            report.append(f"Derrotas significativas: {losses}")
            report.append(f"Score neto: {wins - losses}")
            report.append("")
            
            # Guardar reporte
            with open(f"{model_dir}/perfil_{model.replace(' ', '_')}.txt", 'w', encoding='utf-8') as f:
                f.write('\n'.join(report))
            
            # Visualizaciones del modelo
            self._create_model_visualizations(model, model_dir)
    
    def _create_model_visualizations(self, model, model_dir):
        """Crea visualizaciones específicas para un modelo"""
        
        # 1. Distribución de ECRPS
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Histograma
        axes[0, 0].hist(self.df_all[model], bins=50, alpha=0.7, color='steelblue', edgecolor='black')
        axes[0, 0].axvline(self.df_all[model].mean(), color='red', linestyle='--', 
                          linewidth=2, label=f'Media: {self.df_all[model].mean():.4f}')
        axes[0, 0].axvline(self.df_all[model].median(), color='green', linestyle='--', 
                          linewidth=2, label=f'Mediana: {self.df_all[model].median():.4f}')
        axes[0, 0].set_xlabel('ECRPS')
        axes[0, 0].set_ylabel('Frecuencia')
        axes[0, 0].set_title(f'Distribución de ECRPS - {model}')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Boxplot por escenario
        data_by_scenario = [self.df_all[self.df_all['Escenario'] == esc][model] 
                           for esc in ['Estacionario_Lineal', 'No_Estacionario_Lineal', 'No_Lineal']]
        bp = axes[0, 1].boxplot(data_by_scenario, labels=['Est. Lin.', 'No Est. Lin.', 'No Lin.'], 
                               patch_artist=True)
        for patch, color in zip(bp['boxes'], ['lightblue', 'lightcoral', 'lightgreen']):
            patch.set_facecolor(color)
        axes[0, 1].set_ylabel('ECRPS')
        axes[0, 1].set_title(f'ECRPS por Escenario - {model}')
        axes[0, 1].grid(True, alpha=0.3)
        
        # Rendimiento por paso
        paso_data = []
        for p in sorted(self.df_all['Paso'].unique()):
            subset = self.df_all[self.df_all['Paso'] == p]
            if len(subset) > 0:
                paso_data.append((p, subset[model].mean()))
        
        if paso_data:
            pasos, means = zip(*paso_data)
            axes[1, 0].plot(pasos, means, marker='o', linewidth=2, markersize=8, color='darkblue')
            axes[1, 0].set_xlabel('Paso de Predicción')
            axes[1, 0].set_ylabel('ECRPS Promedio')
            axes[1, 0].set_title(f'Rendimiento por Horizonte - {model}')
            axes[1, 0].grid(True, alpha=0.3)
        
        # Heatmap: Distribución × Varianza
        pivot_data = []
        dist_labels = []
        var_labels = sorted(self.df_all['Varianza'].unique())
        
        for dist in self.df_all['Distribución'].unique():
            row = []
            for var in var_labels:
                subset = self.df_all[(self.df_all['Distribución'] == dist) & 
                                    (self.df_all['Varianza'] == var)]
                if len(subset) > 0:
                    row.append(subset[model].mean())
                else:
                    row.append(np.nan)
            if not all(np.isnan(row)):
                pivot_data.append(row)
                dist_labels.append(dist)
        
        if pivot_data:
            pivot_df = pd.DataFrame(pivot_data, index=dist_labels, columns=var_labels)
            
            sns.heatmap(pivot_df, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=axes[1, 1],
                       cbar_kws={'label': 'ECRPS'})
            axes[1, 1].set_title(f'ECRPS: Distribución × Varianza - {model}')
            axes[1, 1].set_xlabel('Varianza')
            axes[1, 1].set_ylabel('Distribución')
        
        plt.tight_layout()
        plt.savefig(f'{model_dir}/visualizaciones_{model.replace(" ", "_")}.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # 2. Comparación con otros modelos
        fig, ax = plt.subplots(figsize=(12, 8))
        
        means = self.df_all[self.models].mean().sort_values()
        colors = ['red' if m == model else 'steelblue' for m in means.index]
        bars = ax.barh(means.index, means.values, color=colors, alpha=0.7)
        
        # Destacar el modelo actual
        for i, bar in enumerate(bars):
            if means.index[i] == model:
                bar.set_edgecolor('black')
                bar.set_linewidth(3)
        
        ax.set_xlabel('ECRPS Promedio')
        ax.set_title(f'Comparación Global - {model} (Destacado en Rojo)')
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'{model_dir}/comparacion_{model.replace(" ", "_")}.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
    
    def generate_recommendations(self, output_dir):
        """
        8. GENERACIÓN DE RECOMENDACIONES
        """
        print("  - Generando recomendaciones estratégicas...")
        
        recommendations = []
        recommendations.append("="*80)
        recommendations.append("RECOMENDACIONES Y CONCLUSIONES")
        recommendations.append("="*80)
        recommendations.append("")
        
        # 1. Modelo campeón general
        overall_best = self.df_all[self.models].mean().idxmin()
        overall_worst = self.df_all[self.models].mean().idxmax()
        
        recommendations.append("1. MODELO CAMPEÓN GENERAL")
        recommendations.append("-" * 40)
        recommendations.append(f"Mejor rendimiento promedio: {overall_best}")
        recommendations.append(f"ECRPS: {self.df_all[overall_best].mean():.6f}")
        recommendations.append(f"Desviación Estándar: {self.df_all[overall_best].std():.6f}")
        recommendations.append("")
        recommendations.append(f"Peor rendimiento promedio: {overall_worst}")
        recommendations.append(f"ECRPS: {self.df_all[overall_worst].mean():.6f}")
        recommendations.append("")
        
        # 2. Modelos por escenario
        recommendations.append("2. RECOMENDACIONES POR ESCENARIO")
        recommendations.append("-" * 40)
        
        for escenario in ['Estacionario_Lineal', 'No_Estacionario_Lineal', 'No_Lineal']:
            subset = self.df_all[self.df_all['Escenario'] == escenario]
            if len(subset) > 0:
                best_model = subset[self.models].mean().idxmin()
                best_score = subset[best_model].mean()
                
                recommendations.append(f"\n{escenario}:")
                recommendations.append(f"  Modelo Recomendado: {best_model}")
                recommendations.append(f"  ECRPS Promedio: {best_score:.6f}")
        
        recommendations.append("")
        
        # 3. Modelos por distribución
        recommendations.append("3. RECOMENDACIONES POR DISTRIBUCIÓN DE ERRORES")
        recommendations.append("-" * 40)
        
        for dist in self.df_all['Distribución'].unique():
            subset = self.df_all[self.df_all['Distribución'] == dist]
            if len(subset) > 0:
                best_model = subset[self.models].mean().idxmin()
                best_score = subset[best_model].mean()
                
                recommendations.append(f"\nDistribución {dist}:")
                recommendations.append(f"  Modelo Recomendado: {best_model}")
                recommendations.append(f"  ECRPS Promedio: {best_score:.6f}")
        
        recommendations.append("")
        
        # 4. Modelos más robustos
        recommendations.append("4. MODELOS MÁS ROBUSTOS (MENOR VARIABILIDAD)")
        recommendations.append("-" * 40)
        
        cv_scores = {model: self.df_all[model].std() / self.df_all[model].mean() 
                    for model in self.models if self.df_all[model].mean() != 0}
        cv_sorted = sorted(cv_scores.items(), key=lambda x: x[1])
        
        for i, (model, cv) in enumerate(cv_sorted[:3], 1):
            recommendations.append(f"{i}. {model}: CV = {cv:.4f}")
        
        recommendations.append("")
        
        # 5. Modelos por horizonte
        recommendations.append("5. RECOMENDACIONES POR HORIZONTE DE PREDICCIÓN")
        recommendations.append("-" * 40)
        
        pasos_unicos = sorted(self.df_all['Paso'].unique())
        for paso in [pasos_unicos[0], pasos_unicos[len(pasos_unicos)//2], pasos_unicos[-1]]:
            subset = self.df_all[self.df_all['Paso'] == paso]
            if len(subset) > 0:
                best_model = subset[self.models].mean().idxmin()
                best_score = subset[best_model].mean()
                
                recommendations.append(f"\nPaso {paso}:")
                recommendations.append(f"  Modelo Recomendado: {best_model}")
                recommendations.append(f"  ECRPS Promedio: {best_score:.6f}")
        
        recommendations.append("")
        
        # 6. Estrategia de ensamble
        recommendations.append("6. ESTRATEGIA DE ENSAMBLE SUGERIDA")
        recommendations.append("-" * 40)
        
        # Top 3 modelos complementarios
        top3 = self.df_all[self.models].mean().nsmallest(3)
        recommendations.append("Combinar los siguientes modelos:")
        for i, (model, score) in enumerate(top3.items(), 1):
            recommendations.append(f"{i}. {model} (ECRPS: {score:.6f})")
        
        recommendations.append("")
        recommendations.append("Justificación:")
        recommendations.append("  - Estos modelos muestran el mejor rendimiento promedio")
        recommendations.append("  - Un ensamble puede capturar fortalezas complementarias")
        recommendations.append("  - Reduce el riesgo de seleccionar un modelo subóptimo")
        
        recommendations.append("")
        
        # 7. Modelos con dominancia estadística
        recommendations.append("7. MODELOS CON DOMINANCIA ESTADÍSTICA")
        recommendations.append("-" * 40)
        
        dominance_scores = []
        for model in self.models:
            wins = 0
            for other_model in self.models:
                if model != other_model:
                    errors1 = self.df_all[model].values
                    errors2 = self.df_all[other_model].values
                    _, p_val = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
                    mean_diff = self.df_all[model].mean() - self.df_all[other_model].mean()
                    
                    if p_val < 0.05 and mean_diff < 0:
                        wins += 1
            
            dominance_scores.append((model, wins))
        
        dominance_scores.sort(key=lambda x: x[1], reverse=True)
        
        recommendations.append("Modelos estadísticamente superiores (test Diebold-Mariano):")
        for i, (model, wins) in enumerate(dominance_scores[:5], 1):
            recommendations.append(f"{i}. {model}: {wins} victorias significativas")
        
        recommendations.append("")
        
        # 8. Reglas de decisión
        recommendations.append("8. REGLAS DE DECISIÓN SUGERIDAS")
        recommendations.append("-" * 40)
        recommendations.append("")
        
        # Reglas por escenario
        for escenario in ['Estacionario_Lineal', 'No_Estacionario_Lineal', 'No_Lineal']:
            subset = self.df_all[self.df_all['Escenario'] == escenario]
            if len(subset) > 0:
                top2 = subset[self.models].mean().nsmallest(2)
                
                if escenario == 'Estacionario_Lineal':
                    recommendations.append("SI el proceso es ESTACIONARIO y LINEAL:")
                elif escenario == 'No_Estacionario_Lineal':
                    recommendations.append("SI el proceso es NO ESTACIONARIO y LINEAL:")
                else:
                    recommendations.append("SI el proceso es NO LINEAL:")
                
                recommendations.append(f"  → Primera opción: {top2.index[0]}")
                recommendations.append(f"  → Segunda opción: {top2.index[1]}")
                recommendations.append("")
        
        # Reglas por distribución
        recommendations.append("SI la distribución de errores:")
        for dist in self.df_all['Distribución'].unique():
            subset = self.df_all[self.df_all['Distribución'] == dist]
            if len(subset) > 0:
                best = subset[self.models].mean().idxmin()
                recommendations.append(f"  • Es {dist} → Usar {best}")
        
        recommendations.append("")
        
        # Reglas por varianza
        recommendations.append("SI el nivel de varianza:")
        variances = sorted(self.df_all['Varianza'].unique())
        if len(variances) >= 2:
            low_var = variances[0]
            high_var = variances[-1]
            
            subset_low = self.df_all[self.df_all['Varianza'] == low_var]
            subset_high = self.df_all[self.df_all['Varianza'] == high_var]
            
            best_low = subset_low[self.models].mean().idxmin()
            best_high = subset_high[self.models].mean().idxmin()
            
            recommendations.append(f"  • Es bajo ({low_var}) → Usar {best_low}")
            recommendations.append(f"  • Es alto ({high_var}) → Usar {best_high}")
        
        recommendations.append("")
        
        # 9. Conclusiones finales
        recommendations.append("9. CONCLUSIONES PRINCIPALES")
        recommendations.append("-" * 40)
        recommendations.append("")
        recommendations.append(f"• El modelo {overall_best} muestra el mejor rendimiento general")
        recommendations.append(f"  con ECRPS promedio de {self.df_all[overall_best].mean():.6f}")
        recommendations.append("")
        
        # Análisis de robustez
        most_robust = min(cv_scores.items(), key=lambda x: x[1])[0]
        recommendations.append(f"• El modelo más robusto (menor CV) es {most_robust}")
        recommendations.append("")
        
        # Comparación estacionario vs no estacionario
        est_best = self.df_estacionario[self.models].mean().idxmin()
        no_est_best = self.df_no_estacionario[self.models].mean().idxmin()
        
        if est_best == no_est_best:
            recommendations.append(f"• {est_best} es consistentemente superior en procesos")
            recommendations.append("  estacionarios y no estacionarios")
        else:
            recommendations.append(f"• Para procesos estacionarios: preferir {est_best}")
            recommendations.append(f"• Para procesos no estacionarios: preferir {no_est_best}")
        recommendations.append("")
        
        # Análisis de no linealidad
        nl_best = self.df_no_lineal[self.models].mean().idxmin()
        recommendations.append(f"• Para procesos no lineales: {nl_best} es la mejor opción")
        recommendations.append("")
        
        # Recomendación de ensamble
        recommendations.append("• Se recomienda implementar un ENSAMBLE de los top 3 modelos")
        recommendations.append("  para maximizar robustez y rendimiento")
        recommendations.append("")
        
        # Consideraciones prácticas
        recommendations.append("10. CONSIDERACIONES PRÁCTICAS")
        recommendations.append("-" * 40)
        recommendations.append("")
        recommendations.append("Factores a considerar en la selección:")
        recommendations.append("  1. Costo computacional vs ganancia en precisión")
        recommendations.append("  2. Robustez ante cambios en la distribución de errores")
        recommendations.append("  3. Consistencia a través de horizontes de predicción")
        recommendations.append("  4. Facilidad de interpretación y explicabilidad")
        recommendations.append("  5. Disponibilidad de recursos para implementación")
        recommendations.append("")
        
        # Trade-offs identificados
        recommendations.append("Trade-offs identificados:")
        
        # Mejor vs más robusto
        if overall_best != most_robust:
            recommendations.append(f"  • Rendimiento vs Robustez: {overall_best} (mejor) vs {most_robust} (más robusto)")
        
        # Modelos especializados
        recommendations.append("  • Algunos modelos son especialistas en escenarios específicos")
        recommendations.append("  • Otros modelos son generalistas con buen rendimiento global")
        recommendations.append("")
        
        # Guardar recomendaciones
        with open(f'{output_dir}/8_recomendaciones.txt', 'w', encoding='utf-8') as f:
            f.write('\n'.join(recommendations))
        
        print('\n'.join(recommendations))


# ============================================================================
# CÓDIGO DE EJECUCIÓN PRINCIPAL
# ============================================================================

def main():
    """
    Función principal para ejecutar el análisis completo
    """
    print("\n" + "="*80)
    print("ANÁLISIS COMPREHENSIVO DE MODELOS DE PREDICCIÓN PROBABILÍSTICA")
    print("="*80 + "\n")
    
    # Crear analizador
    try:
        analyzer = ModelPerformanceAnalyzer()
    except FileNotFoundError:
        print("\nERROR: No se encontraron los archivos de datos")
        print("Verifica que existan los siguientes archivos:")
        print("  - ./Datos/estacionario.xlsx")
        print("  - ./Datos/no_estacionario.xlsx")
        print("  - ./Datos/no_lineal.xlsx")
        return
    except Exception as e:
        print(f"\nERROR al cargar datos: {e}")
        import traceback
        traceback.print_exc()
        return
    
    # Ejecutar análisis completo
    output_directory = 'resultados_analisis_completo'
    
    try:
        analyzer.generate_full_report(output_dir=output_directory)
        
        print(f"\n{'='*80}")
        print(f"✓ Análisis completado exitosamente")
        print(f"✓ Todos los resultados guardados en: {output_directory}/")
        print(f"{'='*80}\n")
        
        print("Archivos generados:")
        print("  📊 Análisis de características del DGP")
        print("  📈 Efectos de distribución y varianza")
        print("  🎯 Análisis de horizonte de predicción")
        print("  🔄 Interacciones complejas")
        print("  💪 Métricas de robustez")
        print("  📉 Tests de Diebold-Mariano")
        print("  👤 Perfiles individuales por modelo")
        print("  💡 Recomendaciones estratégicas")
        print("")
        
    except Exception as e:
        print(f"\n❌ ERROR durante el análisis: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()


ANÁLISIS COMPREHENSIVO DE MODELOS DE PREDICCIÓN PROBABILÍSTICA

Cargando datos...
✓ Estacionario: 1320 filas
  Columnas: ['Paso', 'Valores de AR', 'Valores MA', 'Distribución', 'Varianza error', 'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap', 'Mejor Modelo', 'Escenario']
✓ No Estacionario: 840 filas
  Columnas: ['Paso', 'Tipo de Modelo', 'Valores de AR', 'Valores MA', 'Distribución', 'Varianza error', 'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap', 'Mejor Modelo', 'Escenario']
✓ No Lineal: 840 filas
  Columnas: ['Paso', 'Tipo de Modelo', 'Distribución', 'Varianza error', 'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap', 'Mejor Modelo', 'Escenario']
✓ Tipos de datos convertidos
✓ Filas después de limpieza: 2600

✓ Datos combinados: 2600 observaciones totales
✓ Columnas finales: ['Paso', 'Valores de AR'